# PENINGKATAN PERFORMA CLUSTERING PADA LARGE TEXT DATASET MENGGUNAKAN STAMANTIC SPHERICAL K-MEANS

In [ ]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
import json
from nltk.corpus import wordnet
# import wn, wn.taxonomy
from strsimpy import SIFT4
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Data Collection

Mengolah raw txt dataset dan menggabungkannya menjadi satu file dengan format CSV

### Pubmed Dataset

In [ ]:
def pubmed_data_collection():
    #looping nama file
    symbol = ['ALZ', 'CAN', 'DBT', 'HIV']
    filesname = []
    docs = []
    for i in range(1, 4001):
        if 1 <= i <= 1000:
            filename = f'{symbol[0]}{i}.txt' 
            filesname.append(filename)
        elif 1001 <= i <= 2000:
          filename = f'{symbol[1]}{i}.txt' 
          filesname.append(filename)
        elif 2001 <= i <= 3000:
          filename = f'{symbol[2]}{i}.txt' 
          filesname.append(filename)
        elif 3001 <= i <= 4000:
          filename = f'{symbol[3]}{i}.txt' 
          filesname.append(filename)
        else:
            print('out of range')

    try:
        #looping tiap file
        index = 0
        for files in filesname:
            file = open(f"D:/text clustering/PUBMED 4K/{files}","r")
            content = file.read().splitlines() #list berisi tiap baris dalam file
            doc = {}
            if 0 <= index <= 999:
                doc['label'] = 'alzheimer'
            elif 1000 <= index <= 1999:
                doc['label'] = 'cancer'
            elif 2000 <= index <= 2999:
                doc['label'] = 'diabetes'
            elif 3000 <= index <= 3999:
                doc['label'] = 'HIV'
            else:
                print('index out of range')
            #looping tiap baris dalam 1 file, input ke list
            lines = []
            for line in content:
                lines.append(line)
            #membuat dict tiap file
            for l in lines:
                doc['title'] = lines[0]
                list_abstract = lines[1:]
                doc['abstract'] = list_abstract[0] #mengeluarkan value dari list abstract biar jadi string
                doc['merged'] = doc['title'] + ' ' + doc['abstract']
            docs.append(doc)
            index += 1
            
        #display data
        docs_df = pd.DataFrame.from_dict(docs)
        # print(docs_df)

        #dataframe to csv
        # docs_df.to_csv('D:/text clustering/pubmed.csv', index=False, encoding='utf-8')

        #check data
        # print(f'Jumlah data null : {docs_df.isnull().sum()}\n')
        # print(f'Total data null : {docs_df.isnull().sum().sum()}\n')
        # print(f'Data Duplikat : {docs_df.duplicated()}\n')
    except IOError as e:
        print(IOError, e)

pubmed_data_collection()

### Scopus Dataset

In [ ]:
def scopus_data_collection():
    #looping nama file
    name = ['CONCRETE-2015-', 'HYPERACTIVITY-2015-', 'INVESTMENT-2015-', 'NEURALNETWORK-2015-', 'PHOTOSYNTHESIS-2015-', 'PROTON-2015-', 'TECTONICPLATES-2015-']
    filesname = []
    docs = []
    for i in range(1, 404): #concrete 403, ter-skip 3. total = 400 data
        if i in range(187,190):
            continue
        else:
            filename = f'{name[0]}{i}'
            filesname.append(filename)
    for i in range(1, 421): #hyperactivity 420, ter-skip 20. total = 400 data
        if ((i==77)or(i in range(79,81))or(i==82)or(i==124)or(i==132)or(i==175)or(i in range(207,209)or(i==210)
            or(i==228)or(i==246)or(i==304)or(i==308)or(i==336)or(i==344)or(i==366)or(i==374)or(i==406)or(i==416))):
            continue
        else:
            filename = f'{name[1]}{i}'
            filesname.append(filename)
    for i in range(1, 404): #investment 403, ter-skip 3. total = 400 data
        if (i==13)or(i==31)or(i==105):
            continue
        else:
            filename = f'{name[2]}{i}'
            filesname.append(filename)
    for i in range(1, 402): #neuralnetwork 401, ter-skip 1. total = 400 data
        if i==324:
            continue
        else:
            filename = f'{name[3]}{i}'
            filesname.append(filename)
    for i in range(1, 406): #photosynthesis 405, ter-skip 5. total = 400 data
        if (i==139)or(i==157)or(i==168)or(i==205)or(i==384):
            continue
        else:
            filename = f'{name[4]}{i}'
            filesname.append(filename)
    for i in range(1, 405): #proton 404, ter-skip 4. total = 400 data
        if (i==304)or(i in range(320,322)or(i==327)):
            continue
        else:
            filename = f'{name[5]}{i}'
            filesname.append(filename)
    for i in range(1, 402): #tectonicplates 401, ter-skip 1. total = 400 data
        if i==54:
            continue
        else:
            filename = f'{name[6]}{i}'
            filesname.append(filename)        
    try:
        #looping tiap file
        index = 0
        for files in filesname:
            file = open(f"D:/text clustering/SCOPUS 2.8K/{files}","r",encoding='utf8') 
            content = file.read().strip().splitlines() #list berisi tiap baris dalam file
            doc = {}
            if 0 <= index <= 399:
                doc['label'] = 'concrete'
            elif 400 <= index <= 799:
                doc['label'] = 'hyperactivity'
            elif 800 <= index <= 1199:
                doc['label'] = 'investment'
            elif 1200 <= index <= 1599:
                doc['label'] = 'neuralnetwork'
            elif 1600 <= index <= 1999:
                doc['label'] = 'photosynthesis'
            elif 2000 <= index <= 2399:
                doc['label'] = 'proton'
            elif 2400 <= index <= 2799:
                doc['label'] = 'tectonicplates'
            else:
                print('index out of range')
            lines = []
            #looping tiap baris dalam 1 file, input ke list
            for line in content:
                lines.append(line)
            #membuat dict tiap file
            for l in lines:
                doc['title'] = lines[0]
                doc['abstract'] = lines[1:] #mengeluarkan value abstract yg berbentuk list
                # if len(list_abstract) > 3:  #ada data yg index 0nya berisi \t, jadi aksesnya langsung dari index 1
                #     doc['abstract'] = list_abstract[1] #mengeluarkan value dari list abstract biar jadi string
                # else:
                #     doc['abstract'] = list_abstract[0]
                doc['merged'] = str(doc['title']) + ' ' + str(doc['abstract'])
            docs.append(doc)
            index += 1

        #display data
        docs_df = pd.DataFrame.from_dict(docs)
        # print(docs_df)

        #dataframe to csv
        # docs_df.to_csv('D:/text clustering/scopus.csv', index=False, encoding='utf-8')

        # check data
        # print(f'Jumlah data null : {docs_df.isnull().sum()}\n')
        # print(f'Total data null : {docs_df.isnull().sum().sum()}\n')
        # print(f'Data Duplikat : {docs_df.duplicated()}\n')
    except IOError as e:
        print(IOError, e)

scopus_data_collection()

## Data Preprocessing

Preprocessing:
- Punctuation Removal
- Case Folding
- Tokenization
- Stopwords Removal

Menghasilkan return berupa list dataset yang telah bebas dari noise

In [ ]:
def preprocessing(text):
    text = str(text)
    punctuation_removal = re.sub(r'\d', '', text)
    punctuation_removal = re.sub(r'[^a-zA-Z\s]', '', punctuation_removal) #punctuation removal
    casefolding = punctuation_removal.casefold() #casefolding
    tokenization = word_tokenize(casefolding) #tokenization, outputnya list
    stopwords_list = set(stopwords.words('english')) #daftar stopword bhs inggris
    filtered = [] #list per kata
    for word in tokenization:
        if word not in stopwords_list:
            filtered.append(word)
    return ' '.join(filtered)

PUBMED DATASET APPLY PREPROCESSING

In [ ]:
pubmed = pd.read_csv("pubmed.csv")
pubmed.loc[:,'merged'] = pubmed['merged'].apply(preprocessing)
pubmed['merged'].to_csv('pubmed_cleaned.csv', index=False,encoding='utf-8')

SCOPUS DATASET APPLY PREPROCESSING

In [ ]:
scopus = pd.read_csv("scopus.csv")
scopus.loc[:,'merged'] = scopus['merged'].apply(preprocessing)
scopus['merged'].to_csv('scopus_cleaned.csv',index=False,encoding='utf-8')

MERGED PUBMED AND SCOPUS DATASET PREPROCESSED

In [ ]:
pubmed = pd.read_csv('pubmed_cleaned.csv')
scopus = pd.read_csv('scopus_cleaned.csv')

dataset = pd.concat([pubmed, scopus], ignore_index=True)
dataset.columns = ['preprocessed']
# dataset.to_csv('Dataset.csv', index=False)

## POS Tagging and Extraction of Nouns Adjective
Menghasilkan return berupa dataframe dengan kolom noun, adjective, dan noun adj

In [ ]:
for i in dataset.iterrows():
    pos_tagging_function = pos_tag(dataset.loc[i[0],'preprocessed'].strip().split(' '))
    nouns_list = []
    adj_list = []
    for tuple_text in pos_tagging_function: #satuan tuple dg index 0 kata, dan index 1 tag
        if ((tuple_text[1]=='NN') or (tuple_text[1]=='NNP') or (tuple_text[1]=='NNS')):
            nouns_list.append(tuple_text[0])
        elif ((tuple_text[1]=='JJ') or (tuple_text[1]=='JJR') or (tuple_text[1]=='JJS')):
            adj_list.append(tuple_text[0])
        else:
            continue
        dataset.loc[i[0], 'noun'] = ' '.join(nouns_list)
        dataset.loc[i[0], 'adjective'] = ' '.join(adj_list)
    dataset.loc[i[0], 'noun adj'] = ' '.join(nouns_list+adj_list)
    break
dataset

In [ ]:
dataset.to_csv('Dataset.csv', index=False, encoding='utf-8')

## Word Sense Disambiguation

1. Mencari kata yang ambigu dan tidak ambigu dengan cara mengecek sinonim dari setiap kata:
   - Jika suatu kata memiliki jumlah sinonim 1, maka kata tersebut tidak ambigu. Kata tersebut akan masuk ke dalam daftar kata tidak ambigu
   - Jika suatu kata memiliki jumlah sinonim > 1, maka kata tersebut ambigu. Kata tersebut akan masuk kedalam daftar kata ambigu

In [ ]:
from nltk.corpus import wordnet
from strsimpy import SIFT4

def AmbiguousAndDisambiguous(text):
    disambiguous_senses = {}
    ambiguous_senses = {}
    for word in text:
        synset = wordnet.synsets(word)
        s = []
        for syn in synset:
            if ((syn.pos() == 'n') or (syn.pos() == 'a')):
                s.append(syn)
        if len(s)==1:
            disambiguous_senses[word] = s[0]
        elif len(s)==2:
            ambiguous_senses[word] = s
        elif len(s)>=3:
            ambiguous_senses[word] = s[0:3]
    return (ambiguous_senses, disambiguous_senses)

2. Menentukan sense yang paling tepat dari daftar kata ambigu menggunakan **Wu Palmer Similarity**
   
$ sim(s_{1}, s_{2}) = \frac{2d+D}{l_{1} +l_{2}+D} $

where:
- $l_{1}, l_{2}$ adalah kedalaman sense dari LCS
- *d* adalah kedalaman LCS dari root node 
- $ D = \frac{1}{sift4(str_{1}, str_{2})}$


**Wu Palmer**:

**LCS**

<img src='lcs.png' width="400" />

1. Least Common Subsumer (LCS) merupakan node induk dari kedua sense. Function find_lcs (in=sense1,sense2) -> return lcs+pathnya
	- if lcs = 0, return none
	- if lcs = 1 & path 1, return lcs+pathnya
	- if lcs = 1 & path > 1, return lcs+pathnya terpanjang
	- if lcs > 1 & path 1, return lcs dg path terpanjang dari sense ke root+pathnya
	- if lcs > 1 & path > 1, return lcs dg path terpanjang dari sense ke root+path terpanjangnya
2. Hitung depth dari lcs ke root
3. Hitung depth dari masing2 s1 dan s2 ke lcs

In [ ]:
def compute_lcs(s1,s2):
    lcs = s1.lowest_common_hypernyms(s2)
    if len(lcs) == 1:
        if len(lcs[0].hypernym_paths()) == 1:
            return lcs[0], len(lcs[0].hypernym_paths())
        elif len(lcs[0].hypernym_paths()) > 1:
            depth = ([len(i) for i in lcs[0].hypernym_paths()])
            choosen_path = lcs[0].hypernym_paths()[depth.index(max(depth))]
            return lcs[0], len(choosen_path)
    elif len(lcs) > 1:
        lcs_list = []
        for i in lcs:
            lcs_list.append([len(j) for j in i.hypernym_paths()])
        choosen_lcs = lcs[lcs_list.index(max(lcs_list))]
        if len(choosen_lcs.hypernym_paths()) == 1:
            return choosen_lcs, len(choosen_lcs.hypernym_paths())
        elif len(choosen_lcs.hypernym_paths()) > 1:
            depth = ([len(i) for i in choosen_lcs.hypernym_paths()])
            choosen_path = choosen_lcs.hypernym_paths()[depth.index(max(depth))]
            return choosen_lcs, len(choosen_path)
    else:
        return 0

In [ ]:
def sense_examples(s1,s2):
    s1_ex = s1.examples()
    s2_ex = s2.examples()
    if not(len(s1_ex)) and not(len(s2_ex)):
        return '', ''
    elif not(len(s1_ex)) and (len(s2_ex)):
        return '', s2_ex[0]
    elif (len(s1_ex)) and not(len(s2_ex)):
        return s1_ex[0], ''
    else:
        return s1_ex[0], s2_ex[0]

def sense_hyponyms(s1,s2):
    s1_hy = s1.hyponyms()
    s2_hy = s2.hyponyms()
    if not(len(s1_hy)) and not(len(s2_hy)): 
        return '', ''
    elif not(len(s1_hy)) and (len(s2_hy)):
        return '', s2_hy[0].definition()
    elif (len(s1_hy)) and not(len(s2_hy)):
        return s1_hy[0].definition(), ''
    else:
        return s1_hy[0].definition(), s2_hy[0].definition()
    
def sense_meronyms(s1,s2):
    s1_me = s1.part_meronyms()
    s2_me = s2.part_meronyms()
    if not(len(s1_me)) and not(len(s2_me)):
        return '', ''
    elif not(len(s1_me)) and (len(s2_me)):
        return '', s2_me[0].definition()
    elif (len(s1_me)) and not(len(s2_me)):
        return s1_me[0].definition(), ''
    else:
        return s1_me[0].definition(), s2_me[0].definition()
    
def sense_holonyms(s1,s2):
    s1_ho = s1.part_holonyms()
    s2_ho = s2.part_holonyms()
    if not(len(s1_ho)) and not(len(s2_ho)):
        return '', ''
    elif not(len(s1_ho)) and (len(s2_ho)):
        return '', s2_ho[0].definition()
    elif (len(s1_ho)) and not(len(s2_ho)):
        return s1_ho[0].definition(), ''
    else:
        return s1_ho[0].definition(), s2_ho[0].definition()


In [ ]:
def noun_similarity(s1,s2,syn1,syn2):
    lcs, d = compute_lcs(s1,s2)
    if lcs:
        l1 = lcs.shortest_path_distance(s1)
        l2 = lcs.shortest_path_distance(s2)
        s1_ex, s2_ex = sense_examples(s1,s2)
        s1_hy, s2_hy = sense_hyponyms(s1,s2)
        s1_me, s2_me = sense_meronyms(s1,s2)
        s1_ho, s2_ho = sense_holonyms(s1,s2)
        D = SIFT4().distance(s1.definition()+' '+s1_ex+' '+syn1+' '+lcs.definition()+' '+s1_hy+' '+s1_me+' '+s1_ho,s2.definition()+' '+s2_ex+' '+syn2+' '+lcs.definition()+' '+s2_hy+' '+s2_me+' '+s2_ho)
        sim = (2*d+(D**-1))/(l1+l2+(D**-1))
        return sim
    else:
        return 0
    
def adj_similarity(s1,s2,syn1,syn2):
    d = len(s1.hypernym_paths()) + len(s2.hypernym_paths())
    l1 = s1.shortest_path_distance(s1)
    l2 = s2.shortest_path_distance(s2)
    s1_ex, s2_ex = sense_examples(s1,s2)
    D = SIFT4().distance(s1.definition()+' '+s1_ex+' '+syn1,s2.definition()+' '+s2_ex+' '+syn2)
    sim = (2*d+(D**-1))/(l1+l2+(D**-1))
    return sim

In [ ]:
def wsd(amb,disamb):
    disambiguated = []
    a = [s1 for s in amb.values() for s1 in s]
    d = [s2 for s2 in disamb.values()]
    comparison = a + d
    choosen = {}
    for s1_keys in amb: #looping each word's synset
        for s1 in amb[s1_keys]:
            sense_dict = {}
            for s2_keys in disamb:
                for s2 in comparison:
                    syn_rel1 = wordnet.synsets(s1_keys)[0].lemma_names()[0]
                    syn_rel2 = wordnet.synsets(s2_keys)[0].lemma_names()[0]
                    if ((s1.pos() == 'n') and (s2.pos() == 'n')) and (s1 != s2):
                        sim = noun_similarity(s1,s2,syn_rel1,syn_rel2)
                        if sim == 0:
                            sense_dict[s1] = 0
                        sense_dict[s1] = sim
                    elif ((s1.pos() == 'a') and (s2.pos() == 'a') and (s1 != s2)):
                        sim = adj_similarity(s1,s2,syn_rel1,syn_rel2)
                        sense_dict[s1] = sim
                    else:
                        continue
        value = [i for i in sense_dict.values()]
        best = list(filter(lambda x: sense_dict[x] == max(value), sense_dict))
        if len(best) >= 1:
            choosen[s1_keys] = best[0]
        else:
            continue
    disambiguated.append(choosen)
    return disambiguated[0]

In [ ]:
import pandas as pd
from tqdm import tqdm

data = pd.read_csv('Dataset.csv')
texts = data['noun adj'].to_numpy()
senses_docs = []
for text in tqdm(texts):
    sense_doc = {}
    noun_adj_list = text.strip().split(' ')
    amb, disamb = AmbiguousAndDisambiguous(noun_adj_list)
    disambiguated = wsd(amb,disamb)
    print(disambiguated)
    # break
    # word_sense = disamb.copy()
    # word_sense.update(disambiguated)
    # sense_doc['senses'] = word_sense
    # senses_docs.append(sense_doc)

In [ ]:
df_sense = pd.DataFrame.from_dict(senses_docs)
df_sense.to_csv('features.csv', index=False, encoding='utf-8')

## Term Weighting

## Statistical (TF-IDF)

input = set of disambiguated sense
<br>output = map of sense as a keys and tf-idf as a values

$ TF-IDF(s_{i}) = freq(s_{i}) * (log(\frac{|D|+1}{doc\_count(s_{i})+1})+1) $

where

- $freq(s_{i})$ = total sense of a document
- |*D*| = total number of document in corpus
- $doc\_count(s_{i})$ = total number of document that contain term corresponding to sense $(s_{i})$


1. Compute total sense
2. Compute total document
3. Compute total document that contain the term corresponding to sense

In [ ]:
def tf():

## Semantic (Lexical Chain) 

## Bag Of Words Model

### Bag Of All Words (BOAW)

In [ ]:
data = pd.read_csv('Dataset.csv')
vectorizer = CountVectorizer()
for i in data.iterrows():
    X = vectorizer.fit_transform(data.loc[i[0],'preprocessed'].strip().split(' '))
    vectorizer.get_feature_names_out()
    print(X.toarray())
    break

### Bag Of Nouns (BON)

In [ ]:
vectorizer = CountVectorizer()
for i in dataset.iterrows():
    X = vectorizer.fit_transform(dataset.loc[i[0],'noun'].strip().split(' '))

### Bag Of Nouns and Adjective

In [ ]:
vectorizer = CountVectorizer()
for i in dataset.iterrows():
    X = vectorizer.fit_transform(dataset.loc[i[0],'noun adj'].strip().split(' '))

## Text Clustering

### K-Means

### Spherical K-Means

## Performance Metrics

### Silhouette Coefficient

### Purity

### Adjusted Mutual Information (AMI)